In [11]:
import numpy as np
import torch
from torch import nn
import torch.nn.functional as F
import matplotlib.pyplot as plt
import optuna
from optuna.trial import TrialState

In [2]:
with open('final_dataset.txt') as file:
    TEXT = file.read()

print(TEXT[:400])

PRIMER ACTO


ESCENA I

Fanfarria . Entran arriba los tribunos y los senadores; luego, abajo, por
una puerta SATURNINO y sus seguidores y por otra BASIANO y los
suyos, con tambores y estandartes .

SATURNINO: (A sus seguidores .)
Nobles patricios, protectores de mis derechos,
defiendan la verdad de mi causa con las armas;
y ustedes, compatriotas, mis amados seguidores,
aboguen por mi título de suc


In [3]:
DEVICE = torch.device('cuda')
EPOCHS = 50

In [7]:
def get_batches(arr, batch_size, seq_length):
    '''Create a generator that returns batches of size
       batch_size x seq_length from arr.

       Arguments
       ---------
       arr: Array you want to make batches from
       batch_size: Batch size, the number of sequences per batch
       seq_length: Number of encoded chars in a sequence
    '''

    batch_size_total = batch_size * seq_length
    # total number of batches we can make, // integer division, round down
    n_batches = len(arr)//batch_size_total

    # Keep only enough characters to make full batches
    arr = arr[:n_batches * batch_size_total]
    # Reshape into batch_size rows, n. of first row is the batch size, the other lenght is inferred
    arr = arr.reshape((batch_size, -1))

    # iterate through the array, one sequence at a time
    for n in range(0, arr.shape[1], seq_length):
        # The features
        x = arr[:, n:n+seq_length]
        # The targets, shifted by one
        y = np.zeros_like(x)
        try:
            y[:, :-1], y[:, -1] = x[:, 1:], arr[:, n+seq_length]
        except IndexError:
            y[:, :-1], y[:, -1] = x[:, 1:], arr[:, 0]
        yield x, y

In [8]:
def one_hot_encode(arr, n_labels):

    one_hot_array = np.zeros((np.multiply(*arr.shape),n_labels),dtype=np.float32)

    one_hot_array[np.arange(one_hot_array.shape[0]),arr.flatten()] = 1.

    one_hot_array = one_hot_array.reshape((*arr.shape, n_labels))

    return one_hot_array

In [9]:
chars = tuple(set(TEXT))
int2char = dict(enumerate(chars))
char2int = {ch: ii for ii, ch in int2char.items()}

# encode the text
encoded = np.array([char2int[ch] for ch in TEXT])

In [10]:
batches = get_batches(encoded, 8, 50)
x, y = next(batches)

print('x\n', x[:10, :10])
print('\ny\n', y[:10, :10])

x
 [[56 81 74  6 93 81 79 68 91 36]
 [59 28 76 32 79 53 60 76 32 94]
 [79  5 94  2 85 60  4 35 25 47]
 [94 32 76  8 40 94 92  2  5 79]
 [79 25 94 32 79  7 28 62  8 10]
 [38 62 79 62 25 79 28 62 38 94]
 [94 28 58 60 76 79 32 76 79  4]
 [ 8 76  8 47 94 79 68  7 28 47]]

y
 [[81 74  6 93 81 79 68 91 36 73]
 [28 76 32 79 53 60 76 32 94 32]
 [ 5 94  2 85 60  4 35 25 47 28]
 [32 76  8 40 94 92  2  5 79 25]
 [25 94 32 79  7 28 62  8 10 47]
 [62 79 62 25 79 28 62 38 94  8]
 [28 58 60 76 79 32 76 79  4 60]
 [76  8 47 94 79 68  7 28 47 35]]


In [12]:
train_on_gpu = torch.cuda.is_available()
if(train_on_gpu):
    print('Training on GPU!')
else:
    print('No GPU available, training on CPU; consider making n_epochs very small.')

Training on GPU!


In [87]:
class CharRNN(nn.Module):

    def __init__(self, tokens, n_hidden=612, n_layers=4,
                 drop_prob=0.5, lr=0.001,batch_size=64):
        super().__init__()
        self.drop_prob = drop_prob
        self.n_layers = n_layers
        self.n_hidden = n_hidden
        self.lr = lr
        self.batch_size = batch_size
        self.train_on_gpu = torch.cuda.is_available()
        self.epochs_trained=0

        # creating character dictionaries
        self.chars = tokens
        self.int2char = dict(enumerate(self.chars))
        self.char2int = {ch: ii for ii, ch in self.int2char.items()}
        self.encoded = np.array([self.char2int[ch] for ch in TEXT])

    ## TODO: define the LSTM
        self.lstm = nn.LSTM(len(self.chars), n_hidden, n_layers,
                            dropout=drop_prob, batch_first=True)

        ## TODO: define a dropout layer
        self.dropout = nn.Dropout(drop_prob)

        ## TODO: define the final, fully-connected output layer
        self.fc = nn.Linear(n_hidden, len(self.chars))


    def forward(self, x, hidden):
        ''' Forward pass through the network.
            These inputs are x, and the hidden/cell state `hidden`. '''

        ## TODO: Get the outputs and the new hidden state from the lstm
        r_output, hidden = self.lstm(x, hidden)

        ## TODO: pass through a dropout layer
        out = self.dropout(r_output)

        # Stack up LSTM outputs using view
        # you may need to use contiguous to reshape the output
        out = out.contiguous().view(-1, self.n_hidden)

        ## TODO: put x through the fully-connected layer
        out = self.fc(out)



        # return the final output and the hidden state
        return out, hidden


    def init_hidden(self, batch_size):
        ''' Initializes hidden state '''
        # Create two new tensors with sizes n_layers x batch_size x n_hidden,
        # initialized to zero, for hidden state and cell state of LSTM
        weight = next(self.parameters()).data

        if (self.train_on_gpu):
            hidden = (weight.new(self.n_layers, batch_size, self.n_hidden).zero_().cuda(),
                      weight.new(self.n_layers, batch_size, self.n_hidden).zero_().cuda())
        else:
            hidden = (weight.new(self.n_layers, batch_size, self.n_hidden).zero_(),
                      weight.new(self.n_layers, batch_size, self.n_hidden).zero_())

        return hidden
    def model_train(self, epochs=1, seq_length=50, clip=5, val_frac=0.1, print_every=10,print_state=True):
        ''' Training a network

            Arguments
            ---------

            net: CharRNN network
            data: text data to train the network
            epochs: Number of epochs to train
            batch_size: Number of mini-sequences per mini-batch, aka batch size
            seq_length: Number of character steps per mini-batch
            lr: learning rate
            clip: gradient clipping
            val_frac: Fraction of data to hold out for validation
            print_every: Number of steps for printing training and validation loss

        '''
        loss_arr = []
        val_loss_arr = []
        if(self.train_on_gpu):
            self.cuda()

        self.train()

        opt = torch.optim.Adam(self.parameters(), lr=self.lr)
        criterion = nn.CrossEntropyLoss()

        # create training and validation data
        val_idx = int(len(self.encoded)*(1-val_frac))
        data, val_data = self.encoded[:val_idx], self.encoded[val_idx:]



        counter = 0
        n_chars = len(self.chars)
        for e in range(epochs):
            # initialize hidden state
            h = self.init_hidden(self.batch_size)

            for x, y in get_batches(data, self.batch_size, seq_length):
                counter += 1

                # One-hot encode our data and make them Torch tensors
                x = one_hot_encode(x, n_chars)
                inputs, targets = torch.from_numpy(x), torch.from_numpy(y)

                if(self.train_on_gpu):
                    inputs, targets = inputs.cuda(), targets.cuda().type(torch.cuda.LongTensor)

                # Creating new variables for the hidden state, otherwise
                # we'd backprop through the entire training history
                h = tuple([each.data for each in h])

                # zero accumulated gradients
                self.zero_grad()

                # get the output from the model
                output, h = self(inputs, h)

                # calculate the loss and perform backprop
                loss = criterion(output.cuda(), targets.view(self.batch_size*seq_length))
                loss.backward()
                # `clip_grad_norm` helps prevent the exploding gradient problem in RNNs / LSTMs.
                nn.utils.clip_grad_norm_(self.parameters(), clip)
                opt.step()

                # loss stats
                if counter % print_every == 0 and print_state:
                    # Get validation loss
                    val_h = self.init_hidden(self.batch_size)
                    val_losses = []
                    self.eval()
                    for x, y in get_batches(val_data, self.batch_size, seq_length):
                        # One-hot encode our data and make them Torch tensors
                        x = one_hot_encode(x, n_chars)
                        x, y = torch.from_numpy(x).cuda(), torch.from_numpy(y).cuda()

                        # Creating new variables for the hidden state, otherwise
                        # we'd backprop through the entire training history
                        val_h = tuple([each.data for each in val_h])

                        inputs, targets = x, y
                        if(self.train_on_gpu):
                            inputs, targets = inputs.cuda(), targets.cuda()

                        output, val_h = self(inputs, val_h)
                        val_loss = criterion(output, targets.view(self.batch_size*seq_length).type(torch.cuda.LongTensor))

                        val_losses.append(val_loss.item())

                    self.train() # reset to train mode after iterationg through validation data

                    print("Epoch: {}/{}...".format(e+1, self.epochs_trained),
                          "Step: {}...".format(counter),
                          "Loss: {:.4f}...".format(loss.item()),
                          "Val Loss: {:.4f}".format(np.mean(val_losses)))
                #val_loss_arr.append(val_loss.item())
                loss_arr.append(loss.item())
        return loss_arr

In [93]:
from tqdm.notebook import tnrange, tqdm

class Objective:
    def __init__(self):
        self.model = None
        self.loss = 100

    def define_model(self,trial):
        n_layers = trial.suggest_int("n_layers",4,6)
        n_hidden = trial.suggest_int("n_hidden",312,712,step=100)
        drop_prob = trial.suggest_float("drop_prob",0.4,0.6,step=0.05)
        lr = trial.suggest_float("lr",0.001,0.01,step=0.001)
        batch_size = trial.suggest_int("batch_size",16,128,step=16)

        return CharRNN(chars, n_hidden=n_hidden, n_layers=n_layers, drop_prob=drop_prob, lr=lr,batch_size=batch_size)

    def objective(self,trial):
        model = self.define_model(trial).to('cuda')
        loss = 0
        status = 'Current Params:\n'
        #print('Current Params:',end='\r')
        for key, value in trial.params.items():
            status+="    {}: {}".format(key, value) + '\n'
            #print("    {}: {}".format(key, value),end='\r')
        print(status,end='\r')

        for epoch in tnrange(EPOCHS,desc='Internal Epochs'):
            #print(" ABSOLUTE EPOCH:   "+str(epoch))
            #print("     Relative Epochs:    " + str(model.epochs_trained))
            loss_arr= model.model_train(seq_length=160,print_state=False)
            loss = loss_arr[0]
            print("         Current Loss: %f"%loss,end='\r')
            trial.report(loss,epoch)


            if trial.should_prune():
                raise optuna.exceptions.TrialPruned()

        if loss < self.loss:
            self.loss = loss
            self.model = model
        return loss


In [94]:

evaluator = Objective()

study = optuna.create_study(direction='minimize')
study.optimize(evaluator.objective,n_trials=20,show_progress_bar=True)

pruned_trials = study.get_trials(deepcopy=False, states=[TrialState.PRUNED])
complete_trials = study.get_trials(deepcopy=False, states=[TrialState.COMPLETE])

print("Study statistics: ")
print("  Number of finished trials: ", len(study.trials))
print("  Number of pruned trials: ", len(pruned_trials))
print("  Number of complete trials: ", len(complete_trials))

print("Best trial:")
trial = study.best_trial

print("  Value: ", trial.value)

print("  Params: ")
for key, value in trial.params.items():
    print("    {}: {}".format(key, value))

[I 2022-05-29 18:20:12,295] A new study created in memory with name: no-name-363f8683-9283-4ee4-8502-679f2c2a4473
C:\ProgramData\Anaconda3\envs\robo_shakespeare\lib\site-packages\optuna\progress_bar.py:47: ExperimentalWarning: Progress bar is experimental (supported from v1.2.0). The interface can change in the future.
  self._init_valid()


  0%|          | 0/20 [00:00<?, ?it/s]

Current Params:
    n_layers: 6
    n_hidden: 612
    drop_prob: 0.5
    lr: 0.004
    batch_size: 48


Internal Epochs:   0%|          | 0/50 [00:00<?, ?it/s]

[I 2022-05-29 19:03:29,202] Trial 0 finished with value: 3.285987377166748 and parameters: {'n_layers': 6, 'n_hidden': 612, 'drop_prob': 0.5, 'lr': 0.004, 'batch_size': 48}. Best is trial 0 with value: 3.285987377166748.
Current Params:
    n_layers: 6
    n_hidden: 612
    drop_prob: 0.45
    lr: 0.008
    batch_size: 96


...

[I 2022-05-29 21:15:42,009] Trial 17 pruned. 
Current Params:
    n_layers: 5
    n_hidden: 612
    drop_prob: 0.55
    lr: 0.007
    batch_size: 128


Internal Epochs:   0%|          | 0/50 [00:00<?, ?it/s]

[I 2022-05-29 21:16:10,288] Trial 18 pruned. 
Current Params:
    n_layers: 4
    n_hidden: 712
    drop_prob: 0.4
    lr: 0.009000000000000001
    batch_size: 48


Internal Epochs:   0%|          | 0/50 [00:00<?, ?it/s]

[I 2022-05-29 21:17:19,284] Trial 19 pruned. 
Study statistics: 
  Number of finished trials:  20
  Number of pruned trials:  15
  Number of complete trials:  5
Best trial:
  Value:  0.9168450236320496
  Params: 
    n_layers: 4
    n_hidden: 712
    drop_prob: 0.4
    lr: 0.002
    batch_size: 32


In [95]:
best_model = evaluator.model

print(best_model)

CharRNN(
  (lstm): LSTM(98, 712, num_layers=4, batch_first=True, dropout=0.4)
  (dropout): Dropout(p=0.4, inplace=False)
  (fc): Linear(in_features=712, out_features=98, bias=True)
)


n_hidden 612
drop_prob 0.6
lr 0.003

In [96]:
model_shakespeare_tragedias = 'rnn_50_epoch_4-712_batch-32_lr-002.net'

checkpoint = {'n_hidden': best_model.n_hidden,
              'n_layers': best_model.n_layers,
              'state_dict': best_model.state_dict(),
              'tokens': best_model.chars}

with open(model_shakespeare_tragedias, 'wb') as f:
    torch.save(checkpoint, f)

In [98]:
def predict(net, char, h=None, top_k=None):
    ''' Given a character, predict the next character.
        Returns the predicted character and the hidden state.
    '''

    # tensor inputs
    x = np.array([[net.char2int[char]]])
    x = one_hot_encode(x, len(net.chars))
    inputs = torch.from_numpy(x)

    if(train_on_gpu):
        inputs = inputs.cuda()

    # detach hidden state from history
    h = tuple([each.data for each in h])
    # get the output of the model
    out, h = net(inputs, h)

    # get the character probabilities
    # apply softmax to get p probabilities for the likely next character giving x
    p = F.softmax(out, dim=1).data
    if(train_on_gpu):
        p = p.cpu() # move to cpu

    # get top characters
    # considering the k most probable characters with topk method
    if top_k is None:
        top_ch = np.arange(len(net.chars))
    else:
        p, top_ch = p.topk(top_k)
        top_ch = top_ch.numpy().squeeze()

    # select the likely next character with some element of randomness
    p = p.numpy().squeeze()
    char = np.random.choice(top_ch, p=p/p.sum())

    # return the encoded value of the predicted char and the hidden state
    return net.int2char[char], h

In [99]:
def sample(net, size, prime='PRIMER ACTO', top_k=None):

    if(train_on_gpu):
        net.cuda()
    else:
        net.cpu()

    net.eval() # eval mode

    # First off, run through the prime characters
    chars = [ch for ch in prime]
    h = net.init_hidden(1)
    for ch in prime:
        char, h = predict(net, ch, h, top_k=top_k)

    chars.append(char)

    # Now pass in the previous character and get a new one
    for ii in range(size):
        char, h = predict(net, chars[-1], h, top_k=top_k)
        chars.append(char)

    return ''.join(chars)

In [136]:
output = sample(best_model, 10000,top_k=5, prime='¿Qué ta chendo? ¡Sexo,sexo!')
print(output)

¿Qué ta chendo? ¡Sexo,sexo! ¡Ahora que
este es el mal con las piedras que mi antiguo encuentro
para ser servido!

Salen .

ESCENA VII

Un palacio.
Entran LENNOX, ROSS
y AARÓN, cuanto la puerta .

Sale .

COMINIO: ¿Quién es el dolor de su señor?

ANTONIO: Así es ahí tienes.

SEGUNDA: BRUJA ¡Maldito se acerca!

Salen el MÉDICO y CAFIS.

ANTONIO: ¡Ay, estas señal e informes!

SOLDADO: SEGUNDO No, dijo que se alimentará pensarlo.
Santo espeso del cielo, este amor que se la habría sucedido.

LADY: MACBETH ¿Dónde está el modo? ¡Socorro!
¿No es la verdad de mi amigo, como linaje,
según se estrecha en la tercera cabeza? ¿Qué hace, y su amigo?

ROMEO: ¿Qué? ¿Por qué natural donde el mismo
dulce sabiduría en su curso
el poder derribero en mi muerte a las cuerpas? ¿Cómo es un asesinato
que así soy la cama el punto de esos palabras
como un monto, con medio de los cuernos, con las prostiguidas
en la nada, cual caballos camino aquí con la muerte?
¿No te he vivido esos agravios alcanzados
al menos ca

In [133]:
with open('sample_gustavo.txt', 'w') as f:
    f.write(output)